**Nome:** Ana Carolina Pela Tofetti 

**Nome:** Barbara Martins Damasceno 

# Projeto 1

Importando bibliotecas

In [ ]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd


pi = math.pi

Funções utilizadas

In [67]:
#Função para usar o Blob
def blob_detec():
    #Blob
    # Set up the detector with default parameters.
    params = cv2.SimpleBlobDetector_Params()
    # Set blob color (0=black, 255=white)
    params.filterByColor = True
    params.blobColor = 255
    # Filter by Area
    params.filterByArea = False
    # params.minArea = 1
    # #params.maxArea = 20000
    # Filter by Circularity
    params.filterByCircularity = False
    params.minCircularity = 0.9
    #params.maxCircularity = 1
    # Filter by Convexity
    params.filterByConvexity = False
    params.minConvexity = 0.95
    #params.maxConvexity = 0.9
    # Filter by Inertia
    params.filterByInertia = True
    params.minInertiaRatio = 0.97
    #params.maxInertiaRatio = 1
    # Set up the detector with default parameters.
    detector = cv2.SimpleBlobDetector_create(params)
    return detector

def TratamentoImagem(imagem):
    img_hsv = cv2.cvtColor(imagem, cv2.COLOR_BGR2HSV)
    (canal_H, canal_S, canal_V) = cv2.split(img_hsv)
    
    thresh = np.where((((canal_S<160)|(canal_V<80))),255,0).astype(np.uint8)
    
    kernel = np.ones((5,5),np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    #plt.imshow(opening)
    #plt.show()
    
    return opening

#Função para idenficar os contornos da imagem
def IdentificaContorno(imagem):
    contours,hierarchy = cv2.findContours(imagem, 1, 2)
    return contours

#Identifica borda
def IdentificaBorda(imagemtrtd):
    detector = blob_detec()
    KP = detector.detect(imagemtrtd)
    if (len(KP)==1):
        statusBorda = 'ok'
    else:
        statusBorda = 'nok'
    return statusBorda


def testandoTudo(images,esteira_mm, area_min):
    """Tratamento de imagem"""
    img_tratada = TratamentoImagem(images)
    #plt.imshow(opening)
    #plt.show()
    
    """Encontrando os contornos"""
    contornos = IdentificaContorno(img_tratada)
    
    (h,w) = img_tratada.shape
    fig_pos1 = np.zeros((h,w), dtype='uint8')
    
    fig_contor = images
    
    """Realizando testes"""
    for c in contornos:
        hull = cv2.convexHull(c)
        area = cv2.contourArea(c)
        if (area> area_min):
            
            img = cv2.drawContours(img_tratada,[hull],0,(255,255,255),thickness=cv2.FILLED)
            
            fig_contor = img
            
            #Teste de borda
            situ_borda = IdentificaBorda(img_tratada)
            
            if situ_borda=='ok':
                img = cv2.drawContours(fig_pos1,[hull],0,(255,0,0),1)
                area = cv2.contourArea(c)
                areas[i] = area
                equi_diameter = np.sqrt(4*area/np.pi)
                rel_area = area/area_perfeita

                #Encontrando o diâmetro
                (x,y),radius = cv2.minEnclosingCircle(c)
                center = (int(x),int(y))
                radius = int(radius)
                diametros = round(((equi_diameter*esteira_mm)/w),2)

                #Encontrando a relação A/B
                (xE, yE), (MA, ma), angle = cv2.fitEllipse(c)
                diametros_maior = (MA*esteira_mm)/w
                diametros_menor = (ma*esteira_mm)/w
                relacao = round((diametros_menor/diametros_maior),3)
                
                #and (rel_area>0.98)
                if ((diametros>=49.5) and (diametros<=50.5)):
                    diametro_ok = 'ok'

                    if ((relacao>=0.95) and (relacao<=1.05)):
                        relacao_ok = 'ok'
                        tamanho_ok = 'ok'
                        return situ_borda, diametros, diametro_ok, relacao, relacao_ok, tamanho_ok
                    else: 
                        relacao_ok = 'nok'
                        tamanho_ok = 'nok'
                        return situ_borda, diametros, diametro_ok, relacao, relacao_ok, tamanho_ok

                else: 
                    diametro_ok = 'nok'
                    tamanho_ok = 'nok'
                    return situ_borda, diametros, diametro_ok, '-', '-', tamanho_ok
                        
                
            else:
                situ_borda = 'nok'
                return situ_borda,'-','-','-','-','-'
        else:
            situ_borda = 'nok'
            return situ_borda,'-','-','-','-','-'
        
def blob_detec2(imagemtrtd):
    #Blob
    # Set up the detector with default parameters.
    params = cv2.SimpleBlobDetector_Params()
    # Set blob color (0=black, 255=white)
    params.filterByColor = True
    params.blobColor = 255
    # Filter by Area
    params.filterByArea = True
    params.minArea = 2500
    # #params.maxArea = 20000
    # Filter by Circularity
    params.filterByCircularity = False
    #params.minCircularity = 0.9
    #params.maxCircularity = 1
    # Filter by Convexity
    params.filterByConvexity = False
    #params.minConvexity = 0.95
    #params.maxConvexity = 0.9
    # Filter by Inertia
    params.filterByInertia = False
    #params.minInertiaRatio = 0.97
    #params.maxInertiaRatio = 1
    # Set up the detector with default parameters.
    detector = cv2.SimpleBlobDetector_create(params)
    KP = detector.detect(imagemtrtd)
    return KP

## Tarefa 1

Implementado

In [87]:
#path="NOK_borda\*.jpg"
#path="NOK_tamanho_forma\*.jpg"
#path="OK\*.jpg" #Replace with your folder 
path="ALL\*.jpg" 
k=glob.glob(path)

images=[cv2.imread(images) for images in glob.glob(path)]

print(len(images))     #number of images in folders

#plt.figure(figsize=(200, 800)) 

esteira_mm_fotos = 65

areas = [0]*len(images)
diametros = [0]*len(images)
diametros_maiores = [0]*len(images)
diametros_menores = [0]*len(images)
relacao = [0]*len(images)

raio_perfeito = int((25*508)/65)
area_perfeita = pi*(raio_perfeito**2)

borda_ok = ['nok']*len(images)
tamanho_ok = ['-']*len(images)
diametro_ok = ['-']*len(images)
relacao_ok = ['-']*len(images)

for i in range(len(images)):
    
    a,b,c,d,e,f = testandoTudo(images[i],esteira_mm_fotos,99000)
    borda_ok[i] = a
    diametros[i] = b
    diametro_ok[i] = c
    relacao[i] = d
    relacao_ok[i] = e
    tamanho_ok[i] = f
                
    #plt.subplot(1,len(images),i+1)
    #plt.imshow(fig_pos1,cmap='gray')

    
#plt.show()
    
print("Check bordas:", borda_ok)    
print("Check diametros:", diametro_ok)
print("Diametros:", diametros)
print("Check relacao:", relacao_ok)
print("Relação AB:", relacao)
print("Check bordas:", borda_ok) 
print("Check tamanho:", tamanho_ok)

30
Check bordas: ['nok', 'ok', 'nok', 'ok', 'nok', 'ok', 'nok', 'nok', 'ok', 'ok', 'nok', 'nok', 'nok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok']
Check diametros: ['-', 'nok', '-', 'nok', '-', 'ok', '-', '-', 'nok', 'nok', '-', '-', '-', 'nok', 'nok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok']
Diametros: ['-', 48.1, '-', 45.53, '-', 49.66, '-', '-', 48.47, 49.34, '-', '-', '-', 49.47, 49.37, 49.75, 49.66, 49.74, 49.72, 49.79, 49.64, 49.54, 49.64, 49.59, 49.94, 49.81, 49.69, 49.66, 49.58, 49.81]
Check relacao: ['-', '-', '-', '-', '-', 'ok', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok']
Relação AB: ['-', '-', '-', '-', '-', 1.007, '-', '-', '-', '-', '-', '-', '-', '-', '-', 1.004, 1.007, 1.009, 1.006, 1.009, 1.004, 1.002, 1.007, 1.007, 1.005, 1.006, 1.003, 1.006, 1.005, 1.001]
Check bordas: ['

In [88]:
#Criando um dataframe para salvar
data = {'Arquivo':  k,
        'Teste borda': borda_ok,
        'Diametro': diametros,
        'Teste diametro': diametro_ok,
        'Relação AB': relacao,
        'Teste relação AB': relacao_ok,
        'Teste tamanho': tamanho_ok
        }

df = pd.DataFrame(data)
df


,Arquivo,Teste borda,Diametro,Teste diametro,Relação AB,Teste relação AB,Teste tamanho
0,ALL\Fig_NOK_01.jpg,nok,-,-,-,-,-
1,ALL\Fig_NOK_02.jpg,ok,48.1,nok,-,-,nok
2,ALL\Fig_NOK_03.jpg,nok,-,-,-,-,-
3,ALL\Fig_NOK_04.jpg,ok,45.53,nok,-,-,nok
4,ALL\Fig_NOK_05.jpg,nok,-,-,-,-,-
5,ALL\Fig_NOK_06.jpg,ok,49.66,ok,1.007,ok,ok
6,ALL\Fig_NOK_07.jpg,nok,-,-,-,-,-
7,ALL\Fig_NOK_08.jpg,nok,-,-,-,-,-
8,ALL\Fig_NOK_09.jpg,ok,48.47,nok,-,-,nok
9,ALL\Fig_NOK_10.jpg,ok,49.34,nok,-,-,nok


In [89]:
#Criando um TXT com os dados
with open('resultadoTarefa1.txt', 'a') as f:
    dfAsString = df.to_string(header=False, index=False)
    f.write(dfAsString)

## Tarefa 2


In [90]:
borda_ok_video = []
diametros_video = []
diametro_ok_video = []
relacao_video = []
relacao_ok_video = []
tamanho_ok_video = []
esteira_video_mm = 76

cap = cv2.VideoCapture('Video1_Vedacao.mp4')
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_atual = 1
frames_ok = 0
i = 0
 
while (frame_atual < num_frames-1):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_atual)
    
    # leitura sequencia dos frames do video
    ret, frame = cap.read()
    imgtratada = TratamentoImagem(frame)
    
    #Obtendo o tamanho
    (h,w) = imgtratada.shape
    
    posx = int((h/2))
    posy = int((w/2)-100)
    if (imgtratada[posy,posx]==255):
        frames_ok = frames_ok+1
        frame_atual += 30
        #cv2.imshow("Video", imgtratada)
        if (frames_ok%2 == 0):
            cv2.imshow("Video", imgtratada)
            a,b,c,d,e,f = testandoTudo(frame,esteira_video_mm,100)
            borda_ok_video.append(a) 
            diametros_video.append(b)
            diametro_ok_video.append(c)
            relacao_video.append(d)
            relacao_ok_video.append(e)
            tamanho_ok_video.append(f)       
        
        

    
    frame_atual += 47
    
    
    # Press “q” on keyboard to exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
        
print(frames_ok)
print(borda_ok_video)
print(diametros_video)
print(diametro_ok_video)
print(relacao_video)
print(relacao_ok_video)
print(tamanho_ok_video)

48
['ok', 'nok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'nok', 'ok', 'ok', 'ok', 'nok', 'ok', 'ok', 'nok', 'ok', 'ok', 'nok', 'ok', 'nok', 'ok', 'ok', 'nok']
[49.58, '-', 49.54, 48.61, 49.28, 48.66, 49.71, 48.03, '-', 49.54, 49.46, 49.55, '-', 49.32, 49.13, '-', 45.45, 49.63, '-', 49.47, '-', 49.6, 49.44, '-']
['ok', '-', 'ok', 'nok', 'nok', 'nok', 'ok', 'nok', '-', 'ok', 'nok', 'ok', '-', 'nok', 'nok', '-', 'nok', 'ok', '-', 'nok', '-', 'ok', 'nok', '-']
[1.007, '-', 1.007, '-', '-', '-', 1.006, '-', '-', 1.005, '-', 1.003, '-', '-', '-', '-', '-', 1.005, '-', '-', '-', 1.004, '-', '-']
['ok', '-', 'ok', '-', '-', '-', 'ok', '-', '-', 'ok', '-', 'ok', '-', '-', '-', '-', '-', 'ok', '-', '-', '-', 'ok', '-', '-']
['ok', '-', 'ok', 'nok', 'nok', 'nok', 'ok', 'nok', '-', 'ok', 'nok', 'ok', '-', 'nok', 'nok', '-', 'nok', 'ok', '-', 'nok', '-', 'ok', 'nok', '-']


In [93]:
data2 = {'Teste borda': borda_ok_video,
        'Diametro': diametros_video,
        'Teste diametro': diametro_ok_video,
        'Relação AB': relacao_video,
        'Teste relação AB': relacao_ok_video,
        'Teste tamanho': tamanho_ok_video
        }

df2 = pd.DataFrame(data2)
df2

,Teste borda,Diametro,Teste diametro,Relação AB,Teste relação AB,Teste tamanho
0,ok,49.58,ok,1.007,ok,ok
1,nok,-,-,-,-,-
2,ok,49.54,ok,1.007,ok,ok
3,ok,48.61,nok,-,-,nok
4,ok,49.28,nok,-,-,nok
5,ok,48.66,nok,-,-,nok
6,ok,49.71,ok,1.006,ok,ok
7,ok,48.03,nok,-,-,nok
8,nok,-,-,-,-,-
9,ok,49.54,ok,1.005,ok,ok


In [94]:
#Criando um TXT com os dados
with open('resultadoTarefa2.txt', 'a') as f:
    dfAsString = df2.to_string(header=False, index=False)
    f.write(dfAsString)